In [11]:
import sys
sys.path.append('../')

from ctypes import *
import time
import numpy as np

from datetime import datetime

# from XY_Surface_Scanner.src.device_conf.dwfconstants import *
from dataclasses import dataclass
from src.scan_data import ImCont, ContinuousMode, DwfData, ScanParam, Status, PictureData
from src.device_conf.device import Device
from src.device_conf.device_conti import Device_conti
from src.files_operation.operation import File_Operation

Device.Check_device()
Device.Open_device()

Device_conti.Set_start_offset_output()
Device_conti.Set_shift_aqusition()
Device_conti.Set_continuous_sin_output()

In [10]:
# Restart device
Device_conti.Switch_off_output()
Device.Close_ALL()

In [ ]:
class Mode_continuous:
    WAIT_START = 0.2
    state = 0
    old_line = 0
    max_y_lines = 0
    tail = 0
    line_offset = 0
    act = []

if __name__ == "__main__":
    Wait_stop = ( 1 / ContinuousMode.hzAcq[0].value ) * 2
    resolution = ScanParam.resolution
    double_res = resolution * 2
    act = []
    
    now = datetime.now()
    start_time = (int(now.strftime("%M")) * 60)  + int(now.strftime("%S"))
    current_time = 0
    while (current_time < 60):
        
        now = datetime.now()
        current_time = (int(now.strftime("%M")) * 60)  + int(now.strftime("%S")) - start_time
        # print( current_time)
        actual = Device_conti.Get_conti_data()
        act.append(actual)
        print("Actual: ", actual)
        
        for i in range(ContinuousMode.buf_size):
            ContinuousMode.f_ch1[i] = float(ContinuousMode.DataCH1[i])
            ContinuousMode.f_ch2[i] = float(ContinuousMode.DataCH2[i])
        
        if (actual == ContinuousMode.buf_size):
            print("END a good time need to find and glue - FAG")
            ending = ContinuousMode.buf_size
            off = 4000
            point = ending - off
            for i in  range(point, ending - double_res, 1):
                ofA = i
                ofsetC = i + double_res
                if ( (ContinuousMode.v_ch1[0:double_res] == ContinuousMode.f_ch1[ofA : ofsetC])).all():
                    line = int ((ending - i) / double_res) - 1
                    print("Hurraaa:", line ) # i = 15015
                    
                    for y_line in range(line):
                            y_lin = int(y_line + Mode_continuous.old_line)
                            if(y_lin < resolution):
                                # ress_off = resolution + Mode_continuous.line_offset
                                ofA = y_line * double_res + ofsetC
                                ofB = (y_line * double_res) + resolution + ofsetC
                                ofC = (y_line + 1) * double_res + ofsetC
                                print("y: ", y_lin, ", of: ", Mode_continuous.line_offset, "|", ofA, " , ", ofB, " , ", ofC, " | ", ofsetC, "...", Mode_continuous.old_line)
                                PictureData.CH1[y_lin, 0:resolution] = ContinuousMode.f_ch1[ofA : ofB]
                                PictureData.CH2[y_lin, 0:resolution] = ContinuousMode.f_ch2[ofA : ofB]
                                PictureData.CH3[y_lin, 0:resolution] = ContinuousMode.f_ch1[ofB : ofC]
                                PictureData.CH4[y_lin, 0:resolution] = ContinuousMode.f_ch2[ofB : ofC]
                                ContinuousMode.v_ch1[0:double_res] = ContinuousMode.f_ch1[ofA : ofC]
                                ContinuousMode.v_ch2[0:double_res] = ContinuousMode.f_ch2[ofA : ofC]
                            else:
                                break
                            
                    Mode_continuous.old_line = Mode_continuous.old_line + line
                            
        else:
            line = (actual - (actual % double_res)) / double_res
        
            if (line > 0 and line < resolution):
                    
                for y_line in range(int(Mode_continuous.old_line), int(line), 1):
                    y_lin = y_line + Mode_continuous.line_offset
                    if(y_lin < resolution):
                        # ress_off = resolution + Mode_continuous.line_offset
                        ofA = y_line * double_res
                        ofB = (y_line * double_res) + resolution
                        ofC = (y_line + 1) * double_res
                        PictureData.CH1[y_lin, 0:resolution] = ContinuousMode.f_ch1[ofA : ofB]
                        PictureData.CH2[y_lin, 0:resolution] = ContinuousMode.f_ch2[ofA : ofB]
                        PictureData.CH3[y_lin, 0:resolution] = ContinuousMode.f_ch1[ofB : ofC]
                        PictureData.CH4[y_lin, 0:resolution] = ContinuousMode.f_ch2[ofB : ofC]
                        ContinuousMode.v_ch1[0:double_res] = ContinuousMode.f_ch1[ofA : ofC]
                        ContinuousMode.v_ch2[0:double_res] = ContinuousMode.f_ch2[ofA : ofC]
                    else:
                        break
        
            Mode_continuous.old_line = line
        
        time.sleep(Wait_stop)
        
    print ("END")